# Creating a simulations of regular and warm winter with a burn-in phase

## 1. Imports

In [26]:
from scripts.run_experiments import *
from utils.biomaker_utils import *

# 2. Configuration

In [27]:
# General
USE_WANDB = False # Set to True to use Weights and Biases for logging and visualization
MAX_WORKERS = 1 # Number of workers to use for parallel processing (default 1), recommended to keep at 1 due to memory constraints
PICKLE_DIR = "pickles" # Directory to save the pickles

# Burn-in period
BURN_IN_FOLDER = "twenty_year_burn_in" # Folder name for the burn-in period
BURN_IN_CONFIG_NAME = "twenty_year_burn_in" # Config name for the burn-in period
BURN_IN_YEARS = 20 # Number of years to run the burn-in period
BURN_IN_DAYS_PER_YEAR = 365 # Number of days per year for the burn-in period
DAYS_IN_BURN_IN = BURN_IN_YEARS * BURN_IN_DAYS_PER_YEAR # Number of days in the burn-in period, used to calculate the burn-in period length where simulations start from.
EARLY_EXTINCTION_MONTH_COUNT = 6  # Number of months to count as early extinction, default 6, this is used to early stop a burn-in simulation if the population goes extinct

# Simulation period
DAYS_PER_YEAR = 365 # Number of days per year for the simulation period
SIMULATION_YEARS = 5 # Number of years to run the simulation period

# Experiments
SEASON_TYPES = ["warm", "cold"] # Season types for the environment (possible options ["warm", "cold"]), warm = regular season, cold = cold january season
NUM_SIMS = 25 # Number of simulations to run for each experiment (default 25)

Because running the full we will change the number of simulations to 1 and the number of years to 1.
For the burn in period we will change the number of years to 2. 

In [28]:
BURN_IN_YEARS = 2
DAYS_IN_BURN_IN = BURN_IN_YEARS * BURN_IN_DAYS_PER_YEAR
NUM_SIMS = 1
SIMULATION_YEARS = 1

## 3. Configuration Class for simulation seasonal/monthly changes

In [36]:
class SeasonsConfig:
    def __init__(self, name, years, days_in_year, january_air_diffusion_rate=0.03, slight_alteration = False, simulation = 0):
        self.name = name
        self.years = years
        self.days_in_year = days_in_year
        self.january_air_diffusion_rate = january_air_diffusion_rate
        self.simulation = simulation
        self.key = jr.PRNGKey(simulation)
        self.month_params = {
            "January": {
                "Season": "Winter",
                "AIR_DIFFUSION_RATE": self.january_air_diffusion_rate,
                "SOIL_DIFFUSION_RATE": 0.03,
            },
            "February": {
                "Season": "Winter",
                "AIR_DIFFUSION_RATE": 0.04,
                "SOIL_DIFFUSION_RATE": 0.04,
            },
            "March": {
                "Season": "Spring",
                "AIR_DIFFUSION_RATE": 0.07,
                "SOIL_DIFFUSION_RATE": 0.07,
            },
            "April": {
                "Season": "Spring",
                "AIR_DIFFUSION_RATE": 0.075,
                "SOIL_DIFFUSION_RATE": 0.075,
            },
            "May": {
                "Season": "Spring",
                "AIR_DIFFUSION_RATE": 0.08,
                "SOIL_DIFFUSION_RATE": 0.08,
            },
            "June": {
                "Season": "Summer",
                "AIR_DIFFUSION_RATE": 0.1,
                "SOIL_DIFFUSION_RATE": 0.1,
            },
            "July": {
                "Season": "Summer",
                "AIR_DIFFUSION_RATE": 0.11,
                "SOIL_DIFFUSION_RATE": 0.11,
            },
            "August": {
                "Season": "Summer",
                "AIR_DIFFUSION_RATE": 0.1,
                "SOIL_DIFFUSION_RATE": 0.1,
            },
            "September": {
                "Season": "Autumn",
                "AIR_DIFFUSION_RATE": 0.09,
                "SOIL_DIFFUSION_RATE": 0.09,
            },
            "October": {
                "Season": "Autumn",
                "AIR_DIFFUSION_RATE": 0.07,
                "SOIL_DIFFUSION_RATE": 0.07,
            },
            "November": {
                "Season": "Autumn",
                "AIR_DIFFUSION_RATE": 0.05,
                "SOIL_DIFFUSION_RATE": 0.05,
            },
            "December": {
                "Season": "Winter",
                "AIR_DIFFUSION_RATE": 0.035,
                "SOIL_DIFFUSION_RATE": 0.035,
            },
        }
        if slight_alteration:
            # each months air and soil diffusion rate is slightly altered by multiplying by a random number between 0.9 and 1.1
            # normal distribution with mean 1 and std 0.05
            for month in self.month_params:
                self.month_params[month]["AIR_DIFFUSION_RATE"] *= np.random.normal(1, 0.05)
                self.month_params[month]["SOIL_DIFFUSION_RATE"] *= np.random.normal(1, 0.05)
        self.n_frames = int(
            self.days_in_year
            / len(self.month_params)
        )

    out_file = "output/seasons.mp4" # Output file for the video
    ec_id = "pestilence"  # @param ['persistence', 'pestilence', 'collaboration', 'sideways']
    env_width_type = "landscape"  # @param ['wide', 'landscape', 'square', 'petri']
    nutrient_cap = np.asarray([25, 25]) # Cap on nutrient levels in the environment
    specialize_cost = np.asarray([0.028, 0.028]) # Cost of specialization
    max_lifetime = 3650  # 10 years

    frame_height = 150

    # Set soil_unbalance_limit to 0 to reproduce the original environment. Set it to 1/3 for having self-balancing environments (recommended).
    soil_unbalance_limit = 1 / 3  # @param [0, "1/3"] {type:"raw"}

    agent_model = "minimal"  # @param ['minimal', 'extended']
    mutator_type = "basic"  # @param ['basic', 'randomly_adaptive']


    # How many unique programs (organisms) are allowed in the simulation.
    n_max_programs = 25

    # if True, every 50 steps we check whether the agents go extinct. If they did,
    # we replace a seed in the environment.
    replace_if_extinct = False

    # on what FRAME to double speed.
    when_to_double_speed = []
    # on what FRAME to reset speed.
    when_to_reset_speed = []
    fps = 30
    # zoom_sz affects the size of the image. If this number is not even, the resulting
    # video *may* not be supported by all renderers.
    zoom_sz = 4

    # how many steps per frame we start with. This gets usually doubled many times
    # during the simulation.
    # In the article, we usually use 2 or 4 as the starting value, sometimes 1.
    steps_per_frame = 1

    ### Evaluation ###
    what_to_evaluate = "extracted"  # @param ["initialization", "extracted"]
    n_eval_steps = 100
    n_eval_reps = 1
    eval_key = jr.PRNGKey(123)

## 4. Function for running burn-in phase(s)

In [ ]:
def run_burn_in_simulation(sim):
    pickle_path = os.path.join(PICKLE_DIR, f"burn_in_env_sim_{sim}.pkl")

    # Load the burn-in environment from a pickle file if it exists
    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as f:
            burn_in_env = pickle.load(f)
        logger.info(
            f"Loaded burn-in environment from {pickle_path} for simulation {sim}"
        )
        return burn_in_env

    seed_simulation = sim
    while True:
        try:
            # Configuration for the twenty-year burn-in phase
            twenty_year_burn_in = SeasonsConfig(
                BURN_IN_CONFIG_NAME,
                BURN_IN_YEARS,
                BURN_IN_DAYS_PER_YEAR,
                simulation=seed_simulation,
            )
            env, base_config, env_config, agent_logic, mutator, key, programs = (
                make_configs(twenty_year_burn_in)
            )

            programs, burn_in_env, environment_history = run_seasons(
                env,
                base_config,
                env_config,
                agent_logic,
                mutator,
                key,
                programs,
                days_since_start=0,
                folder=BURN_IN_FOLDER,
                sim=sim,
                fail_on_extinction=True,
            )

            # Save the burn-in environment to a pickle file
            with open(pickle_path, "wb") as f:
                pickle.dump(burn_in_env, f)
            logger.info(
                f"Saved burn-in environment to {pickle_path} for simulation {sim}"
            )

            return burn_in_env

        except ValueError as e:
            logger.warning(f"Early extinction detected: {e}. Retrying with new seed.")
            seed_simulation += 1

## 5. Functions for running experiment simulation(s)

In [37]:

def run_single_simulation(type_of_january, sim, twenty_year_burn_in_env):
    if type_of_january == "warm":
        experiment_config = SeasonsConfig(
            "warm_winter_month",
            SIMULATION_YEARS,
            DAYS_PER_YEAR,
            january_air_diffusion_rate=0.07,
            simulation=sim,
        )
    else:
        experiment_config = SeasonsConfig(
            "regular_winter_month", SIMULATION_YEARS, DAYS_PER_YEAR, simulation=sim
        )

    env, base_config, env_config, agent_logic, mutator, key, programs = make_configs(
        experiment_config
    )
    folder = "warm_winter_month" if type_of_january == "warm" else "basic_seasons"
    programs, env, environment_history = run_seasons(
        twenty_year_burn_in_env,
        base_config,
        env_config,
        agent_logic,
        mutator,
        key,
        programs,
        days_since_start=DAYS_IN_BURN_IN,
        folder=folder,
        sim=sim,
    )

    environment_history.save_results(folder, sim)


def run_experiments(sim, burn_in_env):
    for type_of_january in SEASON_TYPES:
        run_single_simulation(type_of_january, sim, burn_in_env)

## 6. Function for actual execution of simulation

In [39]:
def run_seasons(
    env,
    base_config,
    env_config,
    agent_logic,
    mutator,
    key,
    programs,
    days_since_start=0,
    folder="",
    sim=0,
    fail_on_extinction=False,
):
    environment_history = EnvironmentHistory(
        base_config, days_since_start, folder, sim, USE_WANDB
    )
    frame = start_simulation(env, base_config, env_config)
    with media.VideoWriter(
        base_config.out_file, shape=frame.shape[:2], fps=base_config.fps, crf=18
    ) as video:
        step = 0
        for year in range(base_config.years):
            extinction_counter = 0
            for month_name, month_params in base_config.month_params.items():
                step, env, programs, env_history = perform_simulation(
                    env,
                    programs,
                    base_config,
                    month_params,
                    env_config,
                    agent_logic,
                    mutator,
                    key,
                    video,
                    frame,
                    step=step,
                    season=f"{month_name} {year + 1}",
                )
                environment_history.add_all(
                    env_history, month_params["Season"], month_name, year
                )
                agent_count = environment_history.return_agent_count_of_last_env()
                if agent_count == 0:
                    extinction_counter += 1

                if agent_count > 0:
                    extinction_counter = 0

                if extinction_counter >= EARLY_EXTINCTION_MONTH_COUNT:

                    if fail_on_extinction:
                        raise ValueError(
                            f"Early extinction detected in simulation {sim} at year {year} and month {month_name}"
                        )
                    else:
                        logger.info(
                            f"Early extinction detected in simulation {sim} at year {year} and month {month_name}"
                        )

    return programs, env, environment_history



## 7. Start pipeline of running burn-in phases and experiments

In [43]:
burn_in_environments = [None] * NUM_SIMS

os.makedirs('output', exist_ok=True)
# Run burn-in simulations
for sim in range(NUM_SIMS):
    burn_in_environments[sim] = run_burn_in_simulation(sim)

# Run experiments for each simulation
for sim in range(NUM_SIMS):
    run_experiments(sim, burn_in_environments[sim])


2024-05-26 13:44:07,079 - utils_logger - INFO - Loaded burn-in environment from pickles/burn_in_env_sim_0.pkl for simulation 0


BasicAgentLogic.dsm_num_params = 0
BasicAgentLogic.nsl_num_params = 176
BasicAgentLogic.denm_num_params = 80
BasicAgentLogic.excl_num_params = 41
BasicAgentLogic.repr_num_params = 2
BasicAgentLogic.num_params = 299


Current config:
agent_state_size: 2
etd: DefaultTypeDef: {materials_list: ['VOID', 'AIR', 'EARTH', 'IMMOVABLE', 'SUN', 'OUT_OF_BOUNDS'], types: {'VOID': 0, 'AIR': 1, 'EARTH': 2, 'IMMOVABLE': 3, 'SUN': 4, 'OUT_OF_BOUNDS': 5, 'AGENT_UNSPECIALIZED': 6, 'AGENT_ROOT': 7, 'AGENT_LEAF': 8, 'AGENT_FLOWER': 9}, specialization_idxs: {'AGENT_UNSPECIALIZED': 0, 'AGENT_ROOT': 1, 'AGENT_LEAF': 2, 'AGENT_FLOWER': 3}, agent_types: [6 7 8 9], intangible_mats: [0 1], gravity_mats: [2 6 7 8 9], structural_mats: [6 7 8 9], propagate_structure_mats: [2 3 6 7 8 9], agent_spawnable_mats: [0 1 2], structure_decay_mats: [-1 -1  1  0 -1 -1  5  5  5  5], aging_mats: [6 7 8 9], dissipation_rate_per_spec: [[0.5 0.5],  [1.  1. ],  [1.  1. ],  [1.2 1.2]]}
env_state_size: 6
struct_integrity_cap: 200
abso

2024-05-26 13:44:40,597 - utils_logger - INFO - 31 environments added to history. Current history length: 31
2024-05-26 13:44:40,600 - utils_logger - INFO - Plant count in last environment: 23

2024-05-26 13:44:49,915 - utils_logger - INFO - 31 environments added to history. Current history length: 62
2024-05-26 13:44:49,918 - utils_logger - INFO - Plant count in last environment: 24



KeyboardInterrupt: 

The result is located under output/seasons.mp4!